Process the raw Ausgrid solar data to a more usable format.<br>
Inspired by: https://github.com/pierre-haessig/ausgrid-solar-data

In [1]:
import json
import pandas as pd

In [2]:
fname = 'solar.csv' # 2012-2013 Solar home electricity data v2.csv
d_raw = pd.read_csv(fname, skiprows=1,
                    parse_dates=['date'], dayfirst=True,
                    na_filter=False, dtype={'Row Quality': str})

In [3]:
d_raw.head()

,Customer,Generator Capacity,Postcode,Consumption Category,date,0:30,1:00,1:30,2:00,2:30,...,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00,Row Quality
0,1,3.78,2076,CL,2012-07-01,1.250,1.250,1.250,1.263,0.131,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.081,
1,1,3.78,2076,GC,2012-07-01,0.855,0.786,0.604,0.544,0.597,...,0.374,0.447,0.549,0.136,0.288,0.181,0.651,0.090,0.068,
2,1,3.78,2076,GG,2012-07-01,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,
3,1,3.78,2076,CL,2012-07-02,1.250,1.250,1.125,0.000,0.925,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.069,
4,1,3.78,2076,GC,2012-07-02,0.309,0.082,0.059,0.097,0.290,...,0.353,0.464,0.229,0.811,0.222,0.306,1.034,0.136,0.067,


#### Pick suitable customers for Env

In [4]:
# Group by postcode for some consistency
postcode_counts = d_raw['Postcode'].value_counts()
print("Top 5 postcodes with most samples:")
print(postcode_counts.head(5))

Top 5 postcodes with most samples:
Postcode
2259    27862
2261    18615
2290    11997
2262     7665
2093     7300
Name: count, dtype: int64


In [5]:
postcode = 2261

d_post = d_raw[d_raw['Postcode'] == postcode].copy()
d_post_cust = d_post['Customer'].unique().tolist()
print(f"Customers in postcode {postcode}: {len(d_post_cust)}")
print(d_post_cust)

Customers in postcode 2261: 21
[5, 13, 28, 50, 58, 61, 69, 70, 72, 86, 90, 127, 158, 165, 224, 246, 266, 276, 286, 292, 297]


In [6]:
for id in d_post_cust:
    
    d_customer = d_raw[d_raw['Customer'] == id]

    d_pv = d_customer[d_customer['Consumption Category'] == 'GG'].to_numpy()[:,5:53].flatten()
    d_d = d_customer[d_customer['Consumption Category'] == 'GC'].to_numpy()[:,5:53].flatten()

    print(f"Customer {id}")
    print(f"Generator Capacity: {d_customer['Generator Capacity'].iloc[0]} kWp")
    print(f"Average Generation: {d_pv.mean():.2f} kWh")
    print(f"Average Consumption: {d_d.mean():.2f} kWh")
    # print(d_d.shape, d_pv.shape)
    print("\n")

Customer 5
Generator Capacity: 1.0 kWp
Average Generation: 0.07 kWh
Average Consumption: 0.23 kWh


Customer 13
Generator Capacity: 2.22 kWp
Average Generation: 0.17 kWh
Average Consumption: 0.24 kWh


Customer 28
Generator Capacity: 1.0 kWp
Average Generation: 0.07 kWh
Average Consumption: 0.21 kWh


Customer 50
Generator Capacity: 1.02 kWp
Average Generation: 0.07 kWh
Average Consumption: 0.15 kWh


Customer 58
Generator Capacity: 1.0 kWp
Average Generation: 0.09 kWh
Average Consumption: 0.56 kWh


Customer 61
Generator Capacity: 2.0 kWp
Average Generation: 0.15 kWh
Average Consumption: 0.38 kWh


Customer 69
Generator Capacity: 2.04 kWp
Average Generation: 0.17 kWh
Average Consumption: 0.38 kWh


Customer 70
Generator Capacity: 1.0 kWp
Average Generation: 0.08 kWh
Average Consumption: 0.44 kWh


Customer 72
Generator Capacity: 1.02 kWp
Average Generation: 0.07 kWh
Average Consumption: 0.28 kWh


Customer 86
Generator Capacity: 1.02 kWp
Average Generation: 0.09 kWh
Average Consumptio

In [7]:
consumers = [5, 58, 286, 292]
prosumers = [13, 50, 158, 297]

json_data = dict()

for id in sorted(consumers + prosumers):
    
    d_customer = d_raw[d_raw['Customer'] == id]

    d_pv = d_customer[d_customer['Consumption Category'] == 'GG'].to_numpy()[:,5:53].flatten()
    d_d = d_customer[d_customer['Consumption Category'] == 'GC'].to_numpy()[:,5:53].flatten()

    # Aggregate half-hourly data to hourly by summing consecutive pairs
    d_pv = d_pv.reshape(-1, 2).sum(axis=1)
    d_d = d_d.reshape(-1, 2).sum(axis=1)

    json_data[id] = {'prosumer': True if id in prosumers else False,
                     'pv_capacity': d_customer['Generator Capacity'].iloc[0],
                     'start_date': d_customer['date'].min().strftime('%d-%m-%Y'),
                     'end_date': d_customer['date'].max().strftime('%d-%m-%Y'),
                     'pv': d_pv.tolist(),
                     'demand': d_d.tolist()}

# Save to JSON file
with open('ausgrid.json', 'w') as f:
    json.dump(json_data, f, indent=4)